In [1]:
# Povm

In [2]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path
import itertools

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import ProjectedGradientDescentBacktracking, ProjectedGradientDescentBacktrackingOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement, get_z_measurement
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)
from quara.objects.operators import tensor_product

from quara.data_analysis.simulation import SimulationSetting

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
start_all = time.time()

In [5]:
# setup system
e_sys_1 = ElementalSystem(0, get_normalized_pauli_basis())
c_sys_1 = CompositeSystem([e_sys_1])

e_sys_2 = ElementalSystem(1, get_normalized_pauli_basis())
c_sys_2 = CompositeSystem([e_sys_2])

# |+><+|
# state_x0 = get_x0_1q(c_sys)
# |+i><+i|
# state_y0 = get_y0_1q(c_sys)
# |0><0|
# state_z0 = get_z0_1q(c_sys)
# |1><1|
# state_z1 = get_z1_1q(c_sys)

tester_objects = []

# |+><+| |+i><+i| |0><0| |1><1|
func_list = [get_x0_1q, get_y0_1q, get_z0_1q, get_z1_1q]

for i, funcs in enumerate(itertools.product(func_list, func_list)):
    state1 = funcs[0](c_sys_1)
    state2 = funcs[1](c_sys_2)

    state_2qubit = tensor_product(state1, state2)

    tester_objects.append(state_2qubit)

In [6]:
true_object = tensor_product( get_z_measurement(c_sys_1),  get_z_measurement(c_sys_2))

In [7]:
measurement_n = len(true_object.vecs)
seed = 777
povmt = StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed)

In [8]:
num_data = [100, 1000]
n_rep = 10
measurement_n = len(true_object.vecs)  # 測定値の数

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed),
]

para_list = [
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator()
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]

    start = time.time()

    estimation_results = data_analysis.estimate(
        qtomography=qtomography,
        true_object=true_object,
        num_data=num_data,
        estimator=estimator,
        iteration=n_rep,
    )

    # stock settings of this simulation
    simulation_setting = SimulationSetting(
        name=name,
        estimator=estimator
    )
    print(simulation_setting)
    simulation_settings.append(simulation_setting)

    estimation_results_list.append(estimation_results)

    elapsed_time = time.time() - start
    print("elapsed_time: {0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

 30%|███       | 3/10 [00:00<00:00, 21.43it/s]Name: LinearEstimator(True)
Estimator: LinearEstimator
Loss: None
Algo: None
elapsed_time: 0.007916653156280517[min]

  0%|          | 0/10 [00:00<?, ?it/s]Name: LinearEstimator(False)
Estimator: LinearEstimator
Loss: None
Algo: None
elapsed_time: 0.008883293469746907[min]

  0%|          | 0/10 [00:00<?, ?it/s]Name: ProjectedLinearEstimator(True)
Estimator: ProjectedLinearEstimator
Loss: None
Algo: None
elapsed_time: 0.43976674874623617[min]

100%|██████████| 10/10 [00:26<00:00,  2.61s/it]Name: ProjectedLinearEstimator(False)
Estimator: ProjectedLinearEstimator
Loss: None
Algo: None
elapsed_time: 0.43493332465489704[min]




In [9]:
report.export_report("povmt_2qubit.pdf",
    estimation_results_list,
    simulation_settings,
    true_object,
    tester_objects,
    seed=seed,
    computation_time=sum(elapsed_times)
)

  0%|          | 0/10 [00:00<?, ?it/s]​Generating table of computation time ...
​Generating table of tolerance of physicality constraint violation ...
​Generating table of experimental conditions ...
Generating case list ...
Computation time of estimators ...
​​Generating MSE of empirical distributions blocks ...
100%|██████████| 10/10 [00:00<00:00, 9986.44it/s]
​​Generating consictency test blocks ...
​Generating a graph for MSE ...
  LinearEstimator(True)=[0.2533399999999982, 0.029085399999999928]
  LinearEstimator(False)=[0.27094000000000024, 0.030259400000000065]
  ProjectedLinearEstimator(True)=[0.15539404582940533, 0.01883354812565295]
  ProjectedLinearEstimator(False)=[0.15464602493725757, 0.018579593481549257]
{'make_graphs_func': <function make_mses_graphs_estimator at 0x0000016342F8BB70>, 'mse_type': 'estimator', 'estimation_results_list': [[<quara.protocol.qtomography.standard.linear_estimator.LinearEstimationResult object at 0x00000163438DE630>, <quara.protocol.qtomography.